In [5]:
import glob
import re
import numpy as np
from dictances import bhattacharyya
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
from scipy import signal
%matplotlib inline

In [6]:
def find_all_probs(directory='',N = 10):
    all_dirs=glob.glob(directory+"example?*/")
    file_dict={}
    for directory in all_dirs:
        try:
            prob_a_file_list=glob.glob(directory+"prob_a_full_?*.txt")
            prob_a_file=prob_a_file_list[0]

            prob_b_file_list=glob.glob(directory+"prob_b_full_?*.txt")
            prob_b_file=prob_b_file_list[0]

            prob_a_file = open(prob_a_file, "r")
            prob_b_file = open(prob_b_file, "r")

            prob_a_file_content=prob_a_file.read()
            prob_b_file_content=prob_b_file.read()

            prob_a_array=prob_a_file_content[1:-1].split(' ')
            prob_b_array=prob_b_file_content[1:-1].split(' ')

            prob_a_array=[item for item in prob_a_array if item!='']
            prob_b_array=[item for item in prob_b_array if item!='']

            prob_a_array=[float(item.replace('\n','')) for item in prob_a_array]
            prob_b_array=[float(item.replace('\n','')) for item in prob_b_array]

            prob_a_array_padded = np.pad(prob_a_array, (N//2, N-1-N//2), mode='edge')
            prob_b_array_padded = np.pad(prob_b_array, (N//2, N-1-N//2), mode='edge')

            prob_a_array_smooth = np.convolve(prob_a_array_padded, np.ones((N,))/N, mode='valid')
            prob_b_array_smooth = np.convolve(prob_b_array_padded, np.ones((N,))/N, mode='valid')

            prob_a_array_downsampled =  signal.resample(prob_a_array_smooth, 28)
            prob_b_array_downsampled =  signal.resample(prob_b_array_smooth, 28)

            max_a=max(prob_a_array_downsampled)
            index_a=[ind for ind,val in enumerate(prob_a_array_downsampled) if val==max_a][0]
            max_b=prob_b_array_downsampled[index_a]

            freq=prob_a_file_list[0][prob_a_file_list[0].rfind('.')-3:prob_a_file_list[0].rfind('.')]
            f = open(directory+"max_peak_freq"+freq+".txt", "w")
            f.write(str(max_b))
            f.close()

            f = open(directory+"norm_downsampled_spectrum"+freq+".txt", "w")
            f.write(str(prob_a_array_downsampled))
            f.close()
        except Exception as e:
            print("Error in parsing this directory: "+directory)
            print(e)
            continue
    return 0
        

In [7]:
find_all_probs()

0